# 2. naloga

In [30]:
import pandas as pd
import numpy as np
import scipy.special
from scipy.stats import chi2

In [31]:
data = pd.DataFrame(
    {
        "st zlomov": pd.Series(range(6)),
        "st palic": pd.Series([157, 69, 35, 17, 1, 1])
    }
)

In [32]:
data


,st zlomov,st palic
0,0,157
1,1,69
2,2,35
3,3,17
4,4,1
5,5,1


## Točka a)

In [33]:
n = sum(data['st palic'])
X_bar = sum(data['st palic']*data['st zlomov']) / n

p_hat = X_bar/5

In [34]:
def bin_distrib(x, p, m=5):
    return scipy.special.comb(m, x, exact=True) * p**x * (1 - p)**(m - x)

## Točka b)

$$
    H_0: \text{število lomov je porazdeljeno z Bin($5$, $\hat{p}$)}
$$

In [35]:
# za Pearsonov hi kvadrat test zahtevamo, da je minimum po celici vsaj 5,
# zato smo celici 4 in 5 pridružili celici 3.
O = pd.Series(data[data['st palic'] > 5]['st palic'])
O[3] += data['st palic'][4] + data['st palic'][5]

In [36]:
# izračunajmo pričakovane rezultate po celicah, kjer upoštevamo, 
# da smo celici 3 pridružili celici 4 in 5, torej je potrebno prilagoditi
#  E_3 = n * (p_3 + p_4 + p_5) 

E = pd.Series([n * bin_distrib(x, p_hat, 5) for x in range(4)])
E[3] += n * (bin_distrib(4, p_hat, 5) + bin_distrib(5, p_hat, 5))

X2 = sum((O - E)**2 / E)
print(X2)

44.148661018431994


In [37]:
alpha_1 = 0.05
alpha_2 = 0.01

# določimo še prostostne stopnje chi^2 porazdelitve 
# (po novem) imamo 4 celice, en parameter p_hat smo ocenili iz podatkov, 
# zato bomo imeli 2 prostostni stopnji.
print(chi2.ppf(1 - alpha_1, 2))
print(chi2.ppf(1 - alpha_2, 2))

# testna statistika X2 je na našem vzorcu večja, 
# zato ničelno hipotezo zavržemo tako pri stopnji tveganja 5% kot tudi 1%.

5.991464547107979
9.21034037197618


In [38]:
O

0    157
1     69
2     35
3     19
Name: st palic, dtype: int64

In [39]:
E

0    130.086698
1    107.773742
2     35.715195
3      6.424365
dtype: float64

## Točka c)

In [40]:
# dva načina za izračun iste statistike

X = []
for k in range(6):
    X += [k] * data['st palic'][k]

# 1. način
Lambda = np.prod([
    ( (X_bar**x) * ((5 - X_bar)**(5 - x)) ) / ( (x**x) * ((5 - x)**(5 - x)) )
    for x in X]
    )

# 2. način
def aux(x, x_bar):
    if x == 0:
        return (5 - x) * np.log((5 - x_bar) / (5 - x))
    elif x == 5:
        return x * np.log(x_bar/x)
    else:
        return x * np.log(x_bar/x) + (5 - x) * np.log((5 - x_bar) / (5 - x))

logLambda = sum(aux(x, X_bar) for x in X)

print(np.log(Lambda))
print(logLambda)
# vidimo, da data oba načina na naših podatkih isti rezultat

-222.24428374568322
-222.24428374568282


# Točka d)

In [41]:
Lambda = np.prod([((X_bar**x)/(x**x) * (5 - X_bar)**(5 - x)/(5 - x)**(5 - x))**data['st palic'][x] for x in range(6)])

np.log(Lambda)

-222.24428374568322

In [42]:
# ta funkcija bo ustvarila vrednost Bin(m, p) porazdeljene slučajne spremenljivke
def binomial_random_number(m=5, p=p_hat): 
    return np.random.choice(np.arange(0,6), p=[bin_distrib(x,p,m) for x in range(6)])

def generate_sample(n):
    X = []
    for _ in range(n):
        X.append(binomial_random_number(5, p_hat))
    return X

count = 0
for _ in range(1000):
    X = generate_sample(n)
    x_bar = sum(X)/len(X)
    
    logL = sum(aux(x, x_bar) for x in X)
    # logL = (
    #     sum(x * np.log(X_bar/x) for x in X if x != 0) +
    #     sum((5 - x) * np.log((5 - X_bar) / (5 - x)) for x in X if x != 5)
    # )
    # print(logL)
    # print(x_bar)
    
    if logL > logLambda:
        count += 1

count

1000

Vidimo, da so vrednosti testne statistike na simuliranih vzorcih vedno večje od tiste izračunane na naših konkretnih podatkih, torej bomo ničelno hipotezo, da imajo vse palice zlome porazdeljene binomsko z enakimi parametri, zavrgli.